<a href="https://colab.research.google.com/github/kmtsui/hobby_stock/blob/main/read_wsb_comment_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example to read the database created by reddit_wsb_comment_stream.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import existing database
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:////content/drive/MyDrive/my_db/wsb_July13.db',execution_options=dict(stream_results=True))

In [ ]:
# Examine database content
engine.table_names()

In [ ]:
# Module to read database config
! pip install sqlacodegen

In [ ]:
# Output database config
! sqlacodegen sqlite:////content/drive/MyDrive/my_db/wsb_July7.db --tables tickers,wsbcomments,wsbposts

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////content/drive/MyDrive/my_db/wsb_July13.db')

from sqlalchemy.orm import sessionmaker

In [ ]:
# Recreate metadata from above
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Ticker(Base):
    __tablename__ = 'tickers'

    TickerId = Column(Integer, primary_key=True)
    TickerName = Column(NVARCHAR(20), nullable=False)
    Count = Column(Integer, nullable=False)
    CountNeg = Column(Numeric, nullable=False)
    CountNeu = Column(Numeric, nullable=False)
    CountPos = Column(Numeric, nullable=False)
    CountCom = Column(Numeric, nullable=False)


class Wsbpost(Base):
    __tablename__ = 'wsbposts'

    PostId = Column(Integer, primary_key=True)
    WSBPostId = Column(NVARCHAR(20), nullable=False)
    WSBPostname = Column(NVARCHAR(40), nullable=False)
    PostTime = Column(DateTime, nullable=False)


class Wsbcomment(Base):
    __tablename__ = 'wsbcomments'

    CommentId = Column(Integer, primary_key=True)
    Body = Column(NVARCHAR(100), nullable=False)
    WSBCommentId = Column(NVARCHAR(20), nullable=False)
    PostTime = Column(DateTime, nullable=False)
    ScoreNeg = Column(Numeric, nullable=False)
    ScoreNeu = Column(Numeric, nullable=False)
    ScorePos = Column(Numeric, nullable=False)
    ScoreCom = Column(Numeric, nullable=False)
    PostId = Column(ForeignKey('wsbposts.PostId'), nullable=False, index=True)
    TickerId = Column(ForeignKey('tickers.TickerId'), nullable=False, index=True)

    wsbpost = relationship('Wsbpost')
    ticker = relationship('Ticker')



In [ ]:
# function to make query
from IPython.display import display
import pandas as pd
import sqlalchemy
from sqlalchemy import select 

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    for c in query.all():
        print(c.Body)
        print("-------------------------")
    df = get_results(query)
    display(df)
    #sql(query)

In [ ]:
class ThingOne(object):
    def go(self, session, tickerName):
        ticker = session.query(Ticker).filter(Ticker.TickerName==tickerName).first()
        print ("Ticker id =",ticker.TickerId, "Count = ", ticker.Count)
        print ("Neg = ",ticker.CountNeg,"Neu = ",ticker.CountNeu,"Pos = ",ticker.CountPos,"Compound = ",ticker.CountCom)
        print("-------------------------")
        comment = session.query(Wsbcomment).filter(Wsbcomment.TickerId==ticker.TickerId)
        #comment = session.query(Ticker)
        #comment = session.query(Wsbcomment)
        #comment = session.query(Wsbpost)
        #comment_query = select(Wsbcomment).where(Wsbcomment.TickerId == tickerId)
        #results = conn.execute(comment_query)
        #display_results(comment_query)
        display_results(comment)

def make_query(tickerName):
    with engine.connect() as conn:
        conn = conn.execution_options(stream_results=True)
        metadata.reflect(engine)
        Session = sessionmaker(bind=engine)
        with Session() as session:
                with session.begin():
                        ThingOne().go(session,tickerName.upper())

In [ ]:
make_query("TSLA")